In [1]:
# Data set: https://www.kaggle.com/mirichoi0218/insurance
# Method: multiple regression

# To help visualize the difference between simple and multiple regression:
# Simple regression (2d space): We have a 2d coordinage system, where we try to find the best line for all data points.
# Multiple Regression (3d space): We have 3d coordinate system, where we try to find the best plain for all data points.
# Mutiple Regression (xd space): Same principle, but hard to imagine.

In [4]:
from keras import models
from keras import layers
from keras import optimizers
import numpy as np
import pandas as pd

In [5]:
raw_insurance_data = pd.read_csv('insurance.csv', sep=',',header=1).values
print(raw_insurance_data)

[[18 'male' 33.77 ... 'no' 'southeast' 1725.5523]
 [28 'male' 33.0 ... 'no' 'southeast' 4449.462]
 [33 'male' 22.705 ... 'no' 'northwest' 21984.47061]
 ...
 [18 'female' 36.85 ... 'no' 'southeast' 1629.8335]
 [21 'female' 25.8 ... 'no' 'southwest' 2007.945]
 [61 'female' 29.07 ... 'yes' 'northwest' 29141.3603]]
